# Calculating ENSO Using Intake-ESGF

## Overview

In this workflow, we combine topics covered in previous Pythia Foundations and CMIP6 Cookbook content to apply the [Niño 3.4 Index](https://climatedataguide.ucar.edu/climate-data/nino-sst-indices-nino-12-3-34-4-oni-and-tni) to a broader range of datasets. As a refresher of what the ENSO 3.4 index is, please see the following text, which is also included in the [ENSO Xarray](https://foundations.projectpythia.org/core/xarray/enso-xarray.html) content in the Pythia Foundations content.

> Niño 3.4 (5N-5S, 170W-120W): The Niño 3.4 anomalies may be thought of as representing the average equatorial SSTs across the Pacific from about the dateline to the South American coast. The Niño 3.4 index typically uses a 5-month running mean, and El Niño or La Niña events are defined when the Niño 3.4 SSTs exceed +/- 0.4C for a period of six months or more.

> Niño X Index computation: a) Compute area averaged total SST from Niño X region; b) Compute monthly climatology (e.g., 1950-1979) for area averaged total SST from Niño X region, and subtract climatology from area averaged total SST time series to obtain anomalies; c) Smooth the anomalies with a 5-month running mean; d) Normalize the smoothed values by its standard deviation over the climatological period.

![](https://www.ncdc.noaa.gov/monitoring-content/teleconnections/nino-regions.gif)

The previous example in the Pythia Foundations content detailed a single simulation. In this example, we aim to apply this computation more generically across a variety of datasets.

The overall goal of this tutorial is to produce a plot of ENSO data using Xarray and intake-ESGF. The plots will resemble the Oceanic Niño Index plot shown below.

![ONI index plot from NCAR Climate Data Guide](https://climatedataguide.ucar.edu/sites/default/files/styles/extra_large/public/2022-03/indices_oni_2_2_lg.png)

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Xarray](https://foundations.projectpythia.org/core/xarray/xarray-intro.html) | Necessary | |
| [hvPlot Basics](https://hvplot.holoviz.org/getting_started/hvplot.html) | Necessary | Interactive Visualization with hvPlot |
| [Understanding of NetCDF](https://foundations.projectpythia.org/core/data-formats/netcdf-cf.html) | Helpful | Familiarity with metadata structure |
| [Calculating ENSO with Xarray](https://foundations.projectpythia.org/core/xarray/enso-xarray.html) | Neccessary | Understanding of Masking and Xarray Functions |
| Dask | Helpful | |

- **Time to learn**: 30 minutes

## Imports

In [ ]:
import hvplot.xarray
import holoviews as hv
import numpy as np
import hvplot.xarray
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from intake_esgf import ESGFCatalog
import xarray as xr
import cf_xarray
import warnings
warnings.filterwarnings("ignore")

hv.extension("bokeh")

## Access ESGF-hosted CMIP6 Data
We will use the Climate Model Intercomparison Project version 6 (CMIP6) dataset, which is available from the Earth System Grid Federation (ESGF) data servers.

There is a toolkit, `intake-esgf`, we can use to interface with the data servers, making it easier to search for our datasets.

In [ ]:
cat = ESGFCatalog()
cat.search(
        activity_id='CMIP',
        experiment_id=["historical","ssp585"],
        source_id="CESM2",
        variable_id=["tos"],
        member_id='r11i1p1f1',
        table_id="Omon",
    )

### Load into a DataTree
Once we subset for our data, we can load the data into a datatree, which is a nested structure of `xarray.Dataset`s, which include the climate grid cell statistics as well!

In [ ]:
tos_tree = cat.to_datatree()
tos_tree

The nodes in the tree represent two different, grids. We would like to stay on the native model grid, using the `gn` node of the datatree, which represents the model native grid data.

In [ ]:
ds = tos_tree["gn"].to_dataset()
ds

## Calculate ENSO
The calculation is covered in more detail in the Pythia Foundations book, here, we apply the calculation to our datasets!

In [ ]:
def calculate_enso(ds):
    
    # Subset the El Nino 3.4 index region
    dso = ds.where(
    (ds.cf["latitude"] < 5) & (ds.cf["latitude"] > -5) & (ds.cf["longitude"] > 190) & (ds.cf["longitude"] < 240), drop=True
    )
    
    # Calculate the monthly means
    gb = dso.tos.groupby('time.month')
    
    # Subtract the monthly averages, returning the anomalies
    tos_nino34_anom = gb - gb.mean(dim='time')
    
    # Determine the non-time dimensions and average using these
    non_time_dims = set(tos_nino34_anom.dims)
    non_time_dims.remove(ds.tos.cf["T"].name)
    weighted_average = tos_nino34_anom.weighted(ds["areacello"]).mean(dim=list(non_time_dims))
    
    # Calculate the rolling average
    rolling_average = weighted_average.rolling(time=5, center=True).mean()
    std_dev = weighted_average.std()
    return rolling_average / std_dev

In [ ]:
enso_index = calculate_enso(ds).compute()
enso_index

## Visualize ENSO

### Basic Visualization
We can create a basic visualization of the dataset using hvplot!

In [ ]:
enso_index.hvplot(x='time')

### Identify El Niño and La Niña
Including the indices as we showed above is not always the most helpful. We need to add additional context to help the reader understand when we reach El Niño and La Niña, which are helpful thresholds for the wider community to use.

A typical threshold to use is 0.4, which means El Niño occurs when the ENSO 3.4 index is equal to or greater than 0.4, and La Niña occurs when the ENSO 3.4 index is equal to or less than 0.4.

We apply this using the following function.

In [ ]:
def add_enso_thresholds(da, threshold=0.4):
    
    # Conver the xr.DataArray into an xr.Dataset
    ds = da.to_dataset()
    
    # Cleanup the time and use the thresholds
    try:
        ds["time"]= ds.indexes["time"].to_datetimeindex()
    except:
        pass
    ds["tos_gt_04"] = ("time", ds.tos.where(ds.tos >= threshold, threshold).data)
    ds["tos_lt_04"] = ("time", ds.tos.where(ds.tos <= -threshold, -threshold).data)
    
    # Add fields for the thresholds
    ds["el_nino_threshold"] = ("time", np.zeros_like(ds.tos) + threshold)
    ds["la_nina_threshold"] = ("time", np.zeros_like(ds.tos) - threshold)
    
    return ds

In [ ]:
enso_ds = add_enso_thresholds(enso_index)
enso_ds

### Configure a Function to Plot the Data
We will use the `hvplot.area` functionality here, which enables us to shade the area between values. We use the newly added variables in our dataset to help here.

In [ ]:
def plot_enso(ds):
    el_nino = ds.hvplot.area(x="time", y2='tos_gt_04', y='el_nino_threshold', color='red', hover=False)
    el_nino_label = hv.Text(ds.isel(time=40).time.values, 2, 'El Niño').opts(text_color='red',)

    # Create the La Niña area graphs
    la_nina = ds.hvplot.area(x="time", y2='tos_lt_04', y='la_nina_threshold', color='blue', hover=False)
    la_nina_label = hv.Text(ds.isel(time=-40).time.values, -2, 'La Niña').opts(text_color='blue')

    # Plot a timeseries of the ENSO 3.4 index
    enso = ds.tos.hvplot(x='time', line_width=0.5, color='k', xlabel='Year', ylabel='ENSO 3.4 Index')

    # Combine all the plots into a single plot
    return (el_nino_label * la_nina_label * el_nino * la_nina * enso)

In [ ]:
plot_enso(enso_ds)

## Apply to Multiple Datasets
Now that we have the workflow, let's apply this to multiple datasets. We focus here on two different instiutions:
- The National Center for Atmospheric Research (NCAR)
- Model for Interdisciplinary Research on Climate (MIROC)

Both of these modeling centers produced output for CMIP6.


### Setup a Function for Searching and Combining Datasets
We can use the query mentioned previously to configure our search. Here, we parameterize based on the institution id (ex. NCAR, MIROC).

In [ ]:
def search_esgf(institution_id, grid='gn'):
    
    # Search and load the ocean surface temperature (tos)
    cat = ESGFCatalog()
    cat.search(
        activity_id="CMIP",
        experiment_id="historical",
        institution_id=institution_id,
        variable_id=["tos"],
        member_id='r11i1p1f1',
        table_id="Omon",
    )
    try:
        tos_ds = cat.to_datatree()[grid].to_dataset()
    except KeyError:
        tos_ds = cat.to_dataset_dict()["tos"]

    return tos_ds

### Apply the Search and Computations

In [ ]:
ncar_ds = search_esgf("NCAR")
enso_index_ncar = add_enso_thresholds(calculate_enso(ncar_ds).compute())

In [ ]:
miroc_ds = search_esgf("MIROC")
enso_index_miroc = add_enso_thresholds(calculate_enso(miroc_ds).compute())

### Visualize our ENSO Comparison
Now that we have our data, we can plot the comparison, stacking the two together using hvPlot.

In [ ]:
ncar_enso_plot = plot_enso(enso_index_ncar).opts(title=f'NCAR {ncar_ds.attrs["source_id"]} \n Ensemble Member: {ncar_ds.attrs["variant_label"]}')
miroc_enso_plot = plot_enso(enso_index_miroc).opts(title=f'MIROC {miroc_ds.attrs["source_id"]} \n Ensemble Member: {miroc_ds.attrs["variant_label"]}')

(ncar_enso_plot + miroc_enso_plot).cols(1)

## Summary
In this notebook, we searched for and accessed two different CMIP6 datasets hosted through ESGF, calculated the ENSO 3.4 indices for the datasets, and created interactive plots comparing where we see El Niño and La Niña.

### What's next?
We will see some more advanced examples of using the CMIP6 and other data access methods as well as computations

## Resources and references
- [Intake-ESGF Documentation](https://github.com/nocollier/intake-esgf)